# <b style='color:magenta;'> Classificação Multi-label com BERT</b>
* Este Jupyter tem como realizar função realizar o tratamento primário dos dados.

---

Bibliotecas/Módulos

<details>    
<summary>
    <font size="3" color="magenta"><b>Install</b></font>
</summary>
<p>
<ul>
    <li> !pip install sklearn </li>
    <li> !pip install pandas </li>
    <li> !pip install scikit-multilearn </li>
</ul>
</p>

In [1]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

Importando Jupyter para tratamento de texto

In [2]:
%%time
%%capture
%run 00TratadorTexto.ipynb
%run 00Confgs.ipynb

CPU times: user 154 ms, sys: 8.26 ms, total: 162 ms
Wall time: 161 ms


----

## Análise do dataset

Importação dos dados

In [3]:
dados = pd.read_csv(PATH_GERAL + "/Dados/stackoverflow_perguntas.csv")

Analisando amostras aleatórias do dataset.

In [4]:
dados.sample(10)

,Perguntas,Tags
151,Gostaria de saber como faço para imprimir atra...,jquery html
1385,"Pessoal Fiz um cronômetro em JS mas ele ""Treme...",html
5090,"estou fazendo um cadastro via ajax, e estou re...",jquery
345,tenho essa tela: Preciso varrer a tela ao c...,jquery angular
1705,Tenho o seguinte gráfico: CODE CODE C...,jquery
3333,Gostaria de saber se é possível identificar el...,jquery
4842,"Gostaria que não repetisse os valores, e no cl...",jquery
3703,preciso que meu site abra em apenas dispositiv...,html
3301,"Tenho uma lista contendo ""URLs"" para outras pá...",jquery
4041,"Tenho a função, e funciona perfeitamente: COD...",jquery


In [5]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5408 entries, 0 to 5407
Data columns (total 2 columns):
Perguntas    5408 non-null object
Tags         5408 non-null object
dtypes: object(2)
memory usage: 84.6+ KB


> Não existem dados faltantes no dataset. Nenhum preenchimento ou substituiçao será necessária.

In [6]:
dados['Tags'].unique()

array(['node.js', 'jquery', 'html', 'html angular ', 'html ', 'angular',
       'angular ', 'jquery html  ', 'jquery ', 'jquery html',
       'jquery html ', 'html angular', 'angular node.js ', 'html  ',
       'jquery html angular', 'node.js ', 'html jquery', 'html jquery ',
       'jquery angular  ', 'html node.js', 'jquery  ', 'angular node.js',
       'jquery angular', 'html node.js ', 'jquery node.js ', 'angular  ',
       'jquery angular ', 'jquery html angular ', 'node.js html ',
       ' node.js', 'node.js html', 'html angular  ', 'jquery node.js',
       'angular html', 'html angular  node.js', 'jquery html node.js',
       'html angular node.js'], dtype=object)

> Tags únicas dentro do dataset

Pode-se notar que as tags estão todas com a grafia correta, nenhuma correção precisará ser feita.

---

In [7]:
lista_de_tags = list()

for tags in dados['Tags'].unique():
    for tag in tags.split():
        if tag not in lista_de_tags:
            lista_de_tags.append(tag)

Listas de tags

In [8]:
lista_de_tags

['node.js', 'jquery', 'html', 'angular']

* Criando colunas para cada uma das tags, adicionando `1` caso a tag esteja presente e `0` para não. 

In [9]:
def nova_coluna(lista_tags: list, dataframe: pd.DataFrame, nome_tags: str):
    """Cria as colunas referentes às tags no dataset.
    
    :param lista_tags: lista com as tags únicas
    :type lista_tags: list
    :param dataframe: dataset
    :type: pd.Dataframe
    :nome_tags: nome da coluna
    :type: str
    """
    
    for tag in lista_tags:
        coluna = list()
        for linha_tag in dataframe[nome_tags]:
            if tag in linha_tag:
                coluna.append(1)
            else:
                coluna.append(0)
        dataframe[tag] = coluna

In [10]:
nova_coluna(lista_de_tags, dados, "Tags")

In [11]:
dados.sample(10)

,Perguntas,Tags,node.js,jquery,html,angular
4954,É possível pegar um valor do javascript e colo...,html,0,0,1,0
4956,Alguem pode me ajudar como muda essa cor azul ...,jquery html,0,1,1,0
249,Preciso carregar os dados unindo dois id Em u...,node.js,1,0,0,0
2889,"Estamos construindo uma aplicação web híbrida,...",html,0,0,1,0
3764,"Como posso tornar um ion-radio como requerido,...",angular,0,0,0,1
1063,Gostaria de saber como ocultar e mostrar div u...,html,0,0,1,0
2466,"Olá, Eu tenho um backend feito com Spring e eu...",angular,0,0,0,1
350,Fiz um método e gostaria de usar ele como Vali...,angular,0,0,0,1
2434,estou tentando retornar o resultado de uma que...,jquery,0,1,0,0
2248,"Como pode ser visto no código, as variáveis de...",html,0,0,1,0


* Tratamento do texto

In [12]:
dados['Perguntas'][5]

'Vi esse vídeo (link abaixo) e gostaria de construir um elemento que somente aparecesse quando a rolagem atingir o "ponto que toque nele". Creio que deve ser javascript. Se alguém souber ficaria muito grato :D Tentei linkar os arquivos e add a div no meu projeto mas ela simplesmente sumiu.  OBS: Ele mostra o efeito em 3:46  https://www.youtube.com/watch?v=tdgDr_icGdo '

> Além do tratamento padrão de texto será necessário também a remoção dos emojis e das urls.

* Os emojis são dispensáveis já que neste caso não vamos trabalhar com análise de sentimento dentro da pergunta.

In [13]:
tratador = TrataTexto()

In [29]:
dados['text'] = dados['Perguntas'].apply(lambda x: tratador.fix_text(x))

In [30]:
dados['text'][5]

'vi video link abaixo gostaria construir elemento somente aparecesse rolagem atingir ponto toque nele creio deve ser javascript alguem souber ficaria grato d tentei linkar arquivos add div projeto simplesmente sumiu obs mostra efeito 3 46'

---

* Juntando todas as tags em uma única coluna que será passada aos classificadores e adicionando ao dataset.

In [31]:
lista_tags_juntas = list(zip(dados[lista_de_tags[0]],
                             dados[lista_de_tags[1]],
                             dados[lista_de_tags[2]],
                             dados[lista_de_tags[3]]))

In [32]:
dados["label"] = lista_tags_juntas

In [33]:
dados.sample(10)

,Perguntas,Tags,node.js,jquery,html,angular,text,label
3794,Estou tentando fazer um componente simples uti...,angular,0,0,0,1,tentando fazer componente simples utilizando r...,"(0, 0, 0, 1)"
4169,Como limpar o cache do IE através de uma funçã...,jquery,0,1,0,0,limpar cache ie atraves funcao javascript quer...,"(0, 1, 0, 0)"
1881,Primeiramente queria dizer que eu cheguei a le...,jquery html,0,1,1,0,primeiramente queria dizer cheguei ler algumas...,"(0, 1, 1, 0)"
3916,"Bom o que eu estava tentando fazer, era a pági...",html,0,0,1,0,bom tentando fazer pagina home pudesse carrega...,"(0, 0, 1, 0)"
4940,Estou usando o plugin Jquery chosen não realiz...,jquery html,0,1,1,0,usando plugin jquery chosen nao realiza filtro...,"(0, 1, 1, 0)"
2097,"Galera, montei um formulário dinâmico usando J...",jquery html,0,1,1,0,galera montei formulario dinamico usando js de...,"(0, 1, 1, 0)"
2065,"Estou a fazer um pedido para fazer um CODE , e...",node.js,1,0,0,0,fazer pedido fazer code problema fazer pedido ...,"(1, 0, 0, 0)"
4998,"Tenho um argumento booleano em um *ngIf, contu...",angular,0,0,0,1,argumento booleano ngif contudo parece nao fun...,"(0, 0, 0, 1)"
2470,Amigos o que está acontecendo é que está passa...,html,0,0,1,0,amigos acontecendo passando espacos numeros re...,"(0, 0, 1, 0)"
4910,Bom dia. Digamos que eu tenho uma tabela com 1...,jquery,0,1,0,0,bom dia digamos tabela 100 linhas gostaria eac...,"(0, 1, 0, 0)"


---

* Separando dataset de treino e teste

In [34]:
x_treino, x_teste, y_treino, y_teste = train_test_split(dados['text'], dados['label'], test_size = 0.2,
                                                        random_state = 123)

In [35]:
print(RED + x_treino[5])

vi video link abaixo gostaria construir elemento somente aparecesse rolagem atingir ponto toque nele creio deve ser javascript alguem souber ficaria grato d tentei linkar arquivos add div projeto simplesmente sumiu obs mostra efeito 3 46


---

TFIDF

In [36]:
vetorizar = TfidfVectorizer(max_features=5000, max_df=0.85)
vetorizar

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.85, max_features=5000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [37]:
vetorizar.fit(dados.Perguntas)
perguntas_treino_tfidf = vetorizar.transform(x_treino)
perguntas_test_tfidf = vetorizar.transform(x_teste)

In [38]:
perguntas_treino_tfidf.shape, perguntas_test_tfidf.shape

((4326, 5000), (1082, 5000))

In [39]:
dados.head()

,Perguntas,Tags,node.js,jquery,html,angular,text,label
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0,possuo projeto node js porem preciso criar exe...,"(1, 0, 0, 0)"
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0,gostaria fazer testes unitarios node js utiliz...,"(1, 0, 0, 0)"
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0,inverter ordem jquery itera array elementos ne...,"(0, 1, 0, 0)"
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0,pagina onde pretendo utilizar conjunto webwork...,"(0, 0, 1, 0)"
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1,exibir dados retornados firestore diretiva ang...,"(0, 0, 1, 1)"


Selecionando apenas as colunas que serão utilizadas 

In [40]:
features = pd.DataFrame(index=dados.index)

In [41]:
features['text'] = dados['text']
features['label'] = dados['label']

In [42]:
features.sample(5)

,text,label
4346,preciso acessar objeto json array exemplo segu...,"(1, 0, 0, 0)"
4656,desejo fazer cara formulario opcao cadastrar p...,"(0, 1, 0, 0)"
1711,trabalhando mapa brasil svg preciso clicar est...,"(0, 1, 0, 0)"
5026,funcao pega id produto id verifica produto ses...,"(0, 1, 0, 0)"
3469,gostaria saber faco colocar caixa texto input ...,"(0, 0, 1, 0)"


In [28]:
# features.to_csv(PATH_GERAL + '/Dados/features.csv')